<a href="https://colab.research.google.com/github/JunTogo/sample_ai_app/blob/main/section_4/02_webapp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 画像認識アプリ
Streamlitを使い、画像を認識するアプリを作りましょう。  
フレームワークにはPyTorch使い、オリジナルのCNNモデルを読み込んで使用します。

## ●ライブラリのインストール
Streamlit、およびアプリの動作の確認に使用する「ngrok」をインストールします。

In [1]:
!pip install streamlit==1.20.0 --quiet
!pip install pyngrok --quiet

Streamlit、およびngrokをインポートしておきます。  
エラーが発生する場合は、「ランタイム」→「ランタイムを再起動」によりランタイムを再起動し、再びコードセルを上から順に実行しましょう。

In [2]:
import streamlit as st
from pyngrok import ngrok

## ●「モデル」を扱うファイル
画像認識の訓練済みモデルを読み込み、予測を行うコードを「model.py」に書き込みます。  

In [4]:
%%writefile model.py
# 以下を「model.py」に書き込み
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image

classes_ja = ["飛行機", "自動車", "鳥", "猫", "鹿", "犬", "カエル", "馬", "船", "トラック"]
classes_en = ["airplane", "automobile", "bird", "cat", "deer", "dog", "frog", "horse", "ship", "truck"]
n_class = len(classes_ja)
img_size = 32

# CNNのモデル
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 256)
        self.dropout = nn.Dropout(p=0.5)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

def predict(img):
    # モデルへの入力
    img = img.convert("RGB")
    img = img.resize((img_size, img_size))
    transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.0, 0.0, 0.0), (1.0, 1.0, 1.0))  # 平均値を0、標準偏差を1に
                                ])
    img = transform(img)
    x = img.reshape(1, 3, img_size, img_size)

    # 訓練済みモデル
    net = Net()
    net.load_state_dict(torch.load(
        "model_cnn.pth", map_location=torch.device("cpu")
        ))

    # 予測
    net.eval()
    y = net(x)

    # 結果を返す
    y_prob = torch.nn.functional.softmax(torch.squeeze(y))  # 確率で表す
    sorted_prob, sorted_indices = torch.sort(y_prob, descending=True)  # 降順にソート
    return [(classes_ja[idx], classes_en[idx], prob.item()) for idx, prob in zip(sorted_indices, sorted_prob)]

Overwriting model.py


## ●アプリのコード
画像認識アプリのコードを、「app.py」に書き込みます。  
ローカルからアップロード、もしくはWebカメラで撮影した画像ファイルに、何が映っているのかを判定します。  
なお、Webカメラはngrokが発行したURLではセキュリティ上動作しないので、今回は動作を確認できません。

In [5]:
%%writefile app.py
# 以下を「app.py」に書き込み
import streamlit as st
import matplotlib.pyplot as plt
from PIL import Image
from model import predict

st.set_option("deprecation.showfileUploaderEncoding", False)

st.sidebar.title("画像認識アプリ")
st.sidebar.write("オリジナルの画像認識モデルを使って何の画像かを判定します。")

st.sidebar.write("")

img_source = st.sidebar.radio("画像のソースを選択してください。",
                              ("画像をアップロード", "カメラで撮影"))
if img_source == "画像をアップロード":
    img_file = st.sidebar.file_uploader("画像を選択してください。", type=["png", "jpg"])
elif img_source == "カメラで撮影":
    img_file = st.camera_input("カメラで撮影")

if img_file is not None:
    with st.spinner("推定中..."):
        img = Image.open(img_file)
        st.image(img, caption="対象の画像", width=480)
        st.write("")

        # 予測
        results = predict(img)

        # 結果の表示
        st.subheader("判定結果")
        n_top = 3  # 確率が高い順に3位まで返す
        for result in results[:n_top]:
            st.write(str(round(result[2]*100, 2)) + "%の確率で" + result[0] + "です。")

        # 円グラフの表示
        pie_labels = [result[1] for result in results[:n_top]]
        pie_labels.append("others")
        pie_probs = [result[2] for result in results[:n_top]]
        pie_probs.append(sum([result[2] for result in results[n_top:]]))
        fig, ax = plt.subplots()
        wedgeprops={"width":0.3, "edgecolor":"white"}
        textprops = {"fontsize":6}
        ax.pie(pie_probs, labels=pie_labels, counterclock=False, startangle=90,
               textprops=textprops, autopct="%.2f", wedgeprops=wedgeprops)  # 円グラフ
        st.pyplot(fig)

Writing app.py


## ●Authtokenの設定
ngrokで接続するために必要な「Authtoken」を設定します。  
以下のコードの、  
`!ngrok authtoken YourAuthtoken`  
における  
`YourAuthtoken`の箇所を、自分のAuthtokenに置き換えます。  
Authtokenは、ngrokのサイトに登録すれば取得することができます。  
https://ngrok.com/


In [6]:
!ngrok authtoken 2o3DUB2wgkogsbeJs5preakG5Hx_6RvZBoFtuePahwfPSwNHp

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


## ●アプリの起動と動作確認
streamlitの`run`コマンドでアプリを起動します。


In [7]:
!streamlit run app.py &>/dev/null&  # 「&>/dev/null&」により、出力を非表示にしてバックグランドジョブとして実行

ngrokのプロセスを終了した上で、新たにポートを指定して接続します。  
接続の結果、urlを取得できます。  
ngrokの無料プランでは同時に1つのプロセスしか動かせないので、エラーが発生した場合は「ランタイム」→「セッションの管理」で不要なGoogle Colabのセッションを修了しましょう。  

In [8]:
ngrok.kill()  # プロセスの修了
url = ngrok.connect("8501").public_url  # 接続

INFO:pyngrok.ngrok:Opening tunnel named: http-8501-22a98efb-cd37-4cdf-9966-71b4996b1b38
2024-10-30 05:55:46.729 INFO    pyngrok.ngrok: Opening tunnel named: http-8501-22a98efb-cd37-4cdf-9966-71b4996b1b38
INFO:pyngrok.process.ngrok:t=2024-10-30T05:55:46+0000 lvl=info msg="no configuration paths supplied"
2024-10-30 05:55:46.756 INFO    pyngrok.process.ngrok: t=2024-10-30T05:55:46+0000 lvl=info msg="no configuration paths supplied"
INFO:pyngrok.process.ngrok:t=2024-10-30T05:55:46+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
2024-10-30 05:55:46.762 INFO    pyngrok.process.ngrok: t=2024-10-30T05:55:46+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml
INFO:pyngrok.process.ngrok:t=2024-10-30T05:55:46+0000 lvl=info msg="open config file" path=/root/.config/ngrok/ngrok.yml err=nil
2024-10-30 05:55:46.767 INFO    pyngrok.process.ngrok: t=2024-10-30T05:55:46+0000 lvl=info msg="open config file" p

urlを表示し、リンク先でアプリが動作することを確認します。

In [9]:
print(url)

https://73a2-35-185-163-44.ngrok-free.app


## ●requirements.txtの作成
Streamlit Cloudのサーバー上でアプリを動かすために、「requirements.txt」を作成する必要があります。  
このファイルでは、必要なライブラリのバージョンを指定します。  


まずは、アプリでimportするライブラリのバージョンを確認します。

In [10]:
import streamlit
import torch
import torchvision
import PIL
import matplotlib

print("streamlit==" + streamlit.__version__)
print("torch==" + torch.__version__)
print("torchvision==" + torchvision.__version__)
print("Pillow==" + PIL.__version__)
print("matplotlib==" + matplotlib.__version__)

streamlit==1.20.0
torch==2.5.0+cu121
torchvision==0.20.0+cu121
Pillow==10.4.0
matplotlib==3.7.1


上記を参考に、各ライブラリの望ましいバージョンを記述しrequirements.txtに保存します。

In [11]:
with open("requirements.txt", "w") as w:
    w.write("streamlit==1.20.1\n")  # Streamlit Cloud上で動作が確認できたバージョン
    w.write("torch==2.5.0\n")  # Cuda対応は要らないのでcu111は記述しない
    w.write("torchvision==0.20.0\n")  # Cuda対応は要らないのでcu111は記述しない
    w.write("Pillow==10.4.0\n")
    w.write("matplotlib==3.7.1\n")

以下の作成されたファイルをダウンロードして、GitHubのレポジトリにアップしましょう。
* app.py
* model.py
* requirements.txt